### Setup libraries

In [5]:
# pip install annoy # conda install -c conda-forge python-annoy
# pip install compress_fasttext
# pip install python-telegram-bot --upgrade
# pip install pymorphy2
# pip install stop_words

### Import libraries

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:72.5% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd

import io
import re
import time
import unicodedata
from pathlib import Path
from linecache import getline
from tqdm.notebook import tqdm

import annoy
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import compress_fasttext
from gensim.models import FastText, KeyedVectors
from gensim.models.fasttext import FastTextKeyedVectors


from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

import matplotlib.pyplot as plt

In [3]:
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

In [4]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#     raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

### Paths to directories and files

In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [6]:
DRAFT_DATASET_PATH = 'E:/kaggle/mailru/Otvety.txt'
# DRAFT_DATASET_PATH = '/content/gdrive/MyDrive/Colab Notebooks/my_projects/Otvety.txt'

### Loading data

In [7]:
lines = io.open(DRAFT_DATASET_PATH, encoding='UTF-8').read()

### Preprocessing data

In [8]:
def preprocess_text(w):
    w = re.sub(':\)', '', w)
    w = re.sub('[)"]', '', w)
    w = re.sub('<[^>]+>', ' ', w)
    
    w = re.sub('\s*\?\s*\.', '?', w)
    w = re.sub('\s*\!\s*\.', '!', w)
    w = re.sub('\s*\.', '.', w)
    w = re.sub('\.+', '.', w)
    
    w = re.sub('---', 'QUESTION', w)
    
    return w

In [9]:
# print(type(lines))
print(len(lines))

1072593541


In [10]:
NUM_EXAMPLES = 7777777
text = preprocess_text(str(lines[:NUM_EXAMPLES]))
print(text[:1000])


QUESTION
вопрос о ТДВ давно и хорошо отдыхаем ЛИЧНО ВАМ здесь кого советовали завести? 
хомячка. 
мужика, йопаря, собачку и 50 кошек. 
Общение! 
паучка. 
Да пол мне бы памыть! А таг то ни чо. Типа ни каво! 
я тут вообще что бы пообщаться. 
А мне советовали сиси завести. 
Ну, слава богу, мужика завести ещё не советовали А вот сватать к кому только не сватали. 
мне тут советовали завести любовника, мужа и много кошек  приветик. 
QUESTION
Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые. 
меня вобще прикалывает эта тема. 
когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы. 
QUESTION
Что делать, сегодня нашёл 2 миллиона рублей? 
Если это счастье  действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни

In [11]:
print(len(text))

7540254


In [12]:
morpher = MorphAnalyzer()
sw = set(get_stop_words('ru'))

In [13]:
def split_by_sentence(sent):
    sent = text.split('\nQUESTION\n')[1:]
    
    questions = []
    contexts = []

    for se in sent:
        se = se.split('\n')
#         se = [i for i in se if (i not in string.punctuation)]
        se = [morpher.parse(i.lower())[0].normal_form for i in se]
#         se = [i for i in se if i not in sw and i != '']

        questions.append(se[0].strip())
        contexts.append(' '.join([f' {s}' for s in se[1:]]))
    
    return questions, contexts

In [14]:
questions, contexts = split_by_sentence(text)
questions[0], contexts[0]

('вопрос о тдв давно и хорошо отдыхаем лично вам здесь кого советовали завести?',
 ' хомячка.   мужика, йопаря, собачку и 50 кошек.   общение!   паучка.   да пол мне бы памыть! а таг то ни чо. типа ни каво!   я тут вообще что бы пообщаться.   а мне советовали сиси завести.   ну, слава богу, мужика завести ещё не советовали а вот сватать к кому только не сватали.   мне тут советовали завести любовника, мужа и много кошек  приветик. ')

In [15]:
print(len(questions[-1])), print(len(contexts[-1]))

102
2694


(None, None)

In [47]:
df = pd.DataFrame({'questions':questions, 'contexts':contexts})

In [48]:
df.isnull().sum()

questions    0
contexts     0
dtype: int64

In [49]:
df.head()

,questions,contexts
0,вопрос о тдв давно и хорошо отдыхаем лично вам здесь кого советовали завести?,"хомячка. мужика, йопаря, собачку и 50 кошек. общение! паучка. да пол мне бы памыть! а таг то ни чо. типа ни каво! я тут вообще что бы пообщаться. а мне советовали сиси завести. ну, слава богу, мужика завести ещё не советовали а вот сватать к кому только не сватали. мне тут совет..."
1,"как парни относятся к цветным линзам? если у девушки то зеленые глаза, то голубые.","меня вобще прикалывает эта тема. когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы."
2,"что делать, сегодня нашёл 2 миллиона рублей?","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке. такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. можно нарваться на бабушку конечно, которая хотела помоч внуку с покупк..."
3,эбу в двенашке называется итэлма что за эбу?,"эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер. он принимает информацию от многочисленных датчиков, обрабатывает ее по особым алгоритмам и, отталкиваясь от полученных данных, отдает команды исполнительным устройствам системы. это завод. а эбу может и..."
4,академия вампиров. сколько на даный момент частей книги академия вампиров?,"4. охотники и жертвы, ледяной укус, поцелуй тьмы, кровная клятва. на данное время их 6. часть 5- оковы для призрака (духовная связь часть 6-последняя жертва."


### simple-baseline

In [50]:
def normalize_answer(text):
    """Lower text and remove punctuation and extra whitespace."""
    return ' '.join(re.findall(r"\w+", text)).lower()

In [51]:
def sentence_to_word(sentences):
    sentences_in_words = list()
    for sentence in sentences:
        sentences_in_words.append(normalize_answer(sentence).split())
    return sentences_in_words

In [52]:
def text_to_sentence(text):
    sentences = text.split(".")
    return [s.strip() for s in sentences if s.strip() != '']

In [68]:
def get_max_match_sentance(data_row, input_question):
    sentences = text_to_sentence(data_row["contexts"])
    sentences_in_words = sentence_to_word(sentences)
    question_in_words = sentence_to_word([data_row["questions"]])[0]

    max_overlap = None
    max_match_sentance_id = None

    question_words = set(question_in_words)
    for sentance_id in range(len(sentences_in_words)):
        sentence_words = set(sentences_in_words[sentance_id])
        overlap = len(sentence_words.intersection(question_words))
        if max_overlap is None or overlap > max_overlap:
            max_overlap = overlap
            max_match_sentance_id = sentance_id

        return sentences[max_match_sentance_id]

In [69]:
# def text_generator(sentence):
#     result, attention_weights = evaluate(sentence)
#     predicted_sentence = tokenizer_en.decode([i for i in result if i < tokenizer_en.vocab_size - 1])
    
#     print(f'Input: {sentence}')
#     # print(f'Predicted message: {predicted_sentence}')
#     return predicted_sentence

In [81]:
def get_response(input_question):   
    df['predictions'] = None
    for data_ind in df.index.values:
        predicted_sentence = get_max_match_sentance(df.loc[data_ind], input_question)
        df.loc[data_ind,('predictions')] = predicted_sentence
        
        if input_question == predicted_sentence:
            print(f'Input: {input_question}')
            print(f'Predicted message: {predicted_sentence}')
            return predicted_sentence
            
    
#     print(f'Input: {input_question}')
#     # print(f'Predicted message: {predicted_sentence}')
#     return predicted_sentence

In [82]:
get_response('плод добродетели.')

In [57]:
# questions, contexts

In [58]:
# question_in_words

In [59]:
# sentences_in_words

In [61]:
df['predictions'] = None
for data_ind in df.index.values:
    full_sentance = get_max_match_sentance(df.loc[data_ind])
    df.loc[data_ind,('predictions')] = full_sentance

In [83]:
df.head()

,questions,contexts,predictions
0,вопрос о тдв давно и хорошо отдыхаем лично вам здесь кого советовали завести?,"хомячка. мужика, йопаря, собачку и 50 кошек. общение! паучка. да пол мне бы памыть! а таг то ни чо. типа ни каво! я тут вообще что бы пообщаться. а мне советовали сиси завести. ну, слава богу, мужика завести ещё не советовали а вот сватать к кому только не сватали. мне тут совет...",хомячка
1,"как парни относятся к цветным линзам? если у девушки то зеленые глаза, то голубые.","меня вобще прикалывает эта тема. когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы.",меня вобще прикалывает эта тема
2,"что делать, сегодня нашёл 2 миллиона рублей?","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке. такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. можно нарваться на бабушку конечно, которая хотела помоч внуку с покупк...","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке"
3,эбу в двенашке называется итэлма что за эбу?,"эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер. он принимает информацию от многочисленных датчиков, обрабатывает ее по особым алгоритмам и, отталкиваясь от полученных данных, отдает команды исполнительным устройствам системы. это завод. а эбу может и...","эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер"
4,академия вампиров. сколько на даный момент частей книги академия вампиров?,"4. охотники и жертвы, ледяной укус, поцелуй тьмы, кровная клятва. на данное время их 6. часть 5- оковы для призрака (духовная связь часть 6-последняя жертва.",4


In [33]:
def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [34]:
def get_score(df_solution, df_predictions):
    score = {
        'f1':
        np.mean([
            f1_score(prediction, answer) for answer, prediction in zip(df_solution, df_predictions)
        ]),
    }
    return score

In [35]:
# from collections import Counter

In [114]:
# df['answer'] = df['predictions']

In [115]:
# del df['answer']

In [116]:
# if 'answer' in df.columns.values:
#     print(get_score(df['predictions'].values, df['answer'].values))

In [118]:
# df.columns.values

In [119]:
df.head()

,questions,contexts,predictions,answer
0,вопрос о тдв давно и хорошо отдыхаем лично вам здесь кого советовали завести?,"хомячка. мужика, йопаря, собачку и 50 кошек. общение! паучка. да пол мне бы памыть! а таг то ни чо. типа ни каво! я тут вообще что бы пообщаться. а мне советовали сиси завести. ну, слава богу, мужика завести ещё не советовали а вот сватать к кому только не сватали. мне тут совет...",хомячка,хомячка
1,"как парни относятся к цветным линзам? если у девушки то зеленые глаза, то голубые.","меня вобще прикалывает эта тема. когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы.",меня вобще прикалывает эта тема,меня вобще прикалывает эта тема
2,"что делать, сегодня нашёл 2 миллиона рублей?","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке. такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. можно нарваться на бабушку конечно, которая хотела помоч внуку с покупк...","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке"
3,эбу в двенашке называется итэлма что за эбу?,"эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер. он принимает информацию от многочисленных датчиков, обрабатывает ее по особым алгоритмам и, отталкиваясь от полученных данных, отдает команды исполнительным устройствам системы. это завод. а эбу может и...","эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер","эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер"
4,академия вампиров. сколько на даный момент частей книги академия вампиров?,"4. охотники и жертвы, ледяной укус, поцелуй тьмы, кровная клятва. на данное время их 6. часть 5- оковы для призрака (духовная связь часть 6-последняя жертва.",4,4


In [ ]:
def evaluate(inp_sentence):
    start_token = [tokenizer_en.vocab_size]
    end_token = [tokenizer_en.vocab_size + 1]
    
    inp_sentence = start_token + tokenizer_en.encode(inp_sentence) + end_token
    encoder_input = tf.expand_dims(inp_sentence, 0)

    decoder_input = [tokenizer_en.vocab_size]
    output = tf.expand_dims(decoder_input, 0)

    for i in range(MAX_LEN):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, output, False,
            enc_padding_mask, combined_mask, dec_padding_mask
        )
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == tokenizer_en.vocab_size+1:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights

In [ ]:
def text_generator(sentence):
    result, attention_weights = evaluate(sentence)
    predicted_sentence = tokenizer_en.decode([i for i in result if i < tokenizer_en.vocab_size - 1])
    
    print(f'Input: {sentence}')
    # print(f'Predicted message: {predicted_sentence}')
    return predicted_sentence

### Evaluate

In [10]:
# text_generator('Что такое ЭБУ?')

In [11]:
# text_generator('Чем заняться?')

In [12]:
# text_generator('Что приготовить?')

In [ ]:
# text_generator('Что посмотреть?')

In [ ]:
# text_generator('Что послушать?')

In [ ]:
# text_generator('Что почитать?')

In [ ]:
# text_generator('Как подключить интернет?')

In [13]:
# text_generator('Как починить машину?')

### python-telegram-bot

In [114]:
updater = Updater(token='')
dispatcher = updater.dispatcher

In [117]:
def answer(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text=text_generator(update.message.text))
    
answer_handler = MessageHandler(Filters.text, answer)
dispatcher.add_handler(answer_handler)

In [118]:
chat = True

if chat:
    updater.start_polling()
    print('@MyPersonalAssistanBot - online')
else:
    updater.stop()
    print('@MyPersonalAssistanBot - offline')

@MyPersonalAssistanBot - online
